<a id="sql_search"></a>
### SQL Search

**Search across SQL files** for keywords, conditions, and patterns. Designed to assist with auditing, debugging, or investigating SQL logic across large codebases.


In [6]:
import pandas as pd
import os
import glob
import re
# from sql_metadata import Parser

pd.set_option('display.max_colwidth', 120)  # Show full column content

In [7]:
def sql_search(word_1, word_2, method='both',approximate=False,path="queries/*/*.sql", with_comments=False):
    # Validate method
    valid_methods = {'both', 'any', 'word_1', 'word_2','word_1_only'}
    if method not in valid_methods:
        raise ValueError(f"Invalid method. Choose from {valid_methods}.")

    # Find all .sql files in the queries/*/ directory
    sql_files = glob.glob(path,recursive=True) # '../../../**/*.sql' for all the source control
    print(f'searching {len(sql_files)} files')
    matching_files = []

    for sql_file in sql_files:
        with open(sql_file, 'r') as file:
            lines = file.readlines()
            lines = [f'File Name: {sql_file}'] + lines
        matches_infile = []
        
        word_1_found = False
        word_2_found = False

        for line_number, line in enumerate(lines, start=1):
            normalized_line = line.strip().lower()
            
            cleaned_line = re.sub('--.*','',normalized_line) # with no comments
            m={}
            
            # Check for word_1 using regex
            if approximate:
                pattern_1=rf'{re.escape(word_1.lower())}'
                pattern_2=rf'{re.escape(word_2.lower())}'
            else:
                pattern_1=rf'\b{re.escape(word_1.lower())}\b'
                pattern_2=rf'\b{re.escape(word_2.lower())}\b'
                
            if re.search(pattern_1, cleaned_line):
                word_1_found = True
                # matches.append(f"Line {line_number}: {line.strip()} (word_1 match)")
                m['file']=sql_file
                m['line_number']=line_number
                m['line']=line.strip()
                m['match']= word_1
                matches_infile.append(m)

            # Check for word_2 using regex
            if re.search(pattern_2, cleaned_line):
                word_2_found = True
                # matches.append(f"Line {line_number}: {line.strip()} (word_2 match)")
                m['file']=sql_file
                m['line_number']=line_number
                m['line']=line.strip()
                m['match']= word_2
                matches_infile.append(m)

        # Determine if the file meets the criteria
        if method == 'both' and word_1_found and word_2_found:
            matching_files += matches_infile
            
        elif method == 'any' and (word_1_found or word_2_found):
            matching_files += matches_infile
        
        elif method == 'word_1_only' and (word_1_found and not word_2_found):
            matching_files += matches_infile
            
        elif method == 'word_1' and word_1_found:
            matching_files += matches_infile
            
        elif method == 'word_2' and word_2_found:
            matching_files += matches_infile
    
    results = pd.DataFrame(matching_files, columns=['file','line_number','line','match'])
    
    if len(matching_files) == 0:
        print(f'no match!\nfor {method}\n word_1 = {word_1} and word_2 = {word_2}') 
     



    return results

In [ ]:
# ----------------------------------
# Search Configuration
# ----------------------------------

word_1 = "isMigrated"
word_2 = "20245"
method = "any"            # Options: 'both', 'any', 'word_1', 'word_2', 'word_1_only'
path = 'queries/**/*.sql' # Use glob patterns to control search scope
appx = True               # Set to False for strict word-boundary matches

# Run search
results = sql_search(word_1, word_2, method, appx, path)

# ----------------------------------
# Ignore List (optional filtering)
# ----------------------------------

ignorelist = [
    r'queries\manual\v_report1.sql',
    r'queries\manual\v_report2.sql',
    r'queries\manual\v_report3.sql',
]

# Example: apply filters to refine search results
# results = results.query('file in @ignorelist')                         # Include only ignored files
# results = results[results.line.str.contains('\(')]                     # Lines containing parentheses
# results = results[~results.line.str.contains('SUBMITTED_TO_CARRIER')]  # Exclude lines with specific text

# ----------------------------------
# Output & Display
# ----------------------------------

matching_files = results['file'].unique()
print(len(matching_files), 'matching files')
for i in matching_files:
    print(i)

# Export results
results.to_csv('matches.csv', index=False)

# Optional: Highlight "File Name" headers in styled DataFrame
def highlight_filename(s):
    return ['color:#aaaa00;font-weight:bold' if 'File Name' in str(v) else '' for v in s]

# Apply styling in notebook (if needed)
# styled_df = results.style.apply(highlight_filename, subset=['line'])


searching 209 files
9 matching files
queries\reports\temp.sql
queries\reports\v_autopay_profile.sql
queries\reports\v_autopay_to_reimbursement.sql
queries\reports\v_booked_deals_platform.sql
queries\reports\v_company_onboarding.sql
queries\reports\v_ee_roster.sql
queries\reports\v_enrollment_app_qa.sql
queries\reports\v_env_rate_validation.sql
queries\reports\v_ye_reporting_contacts.sql


,file,line_number,line,match
0,queries\reports\temp.sql,54,and bcos.status_value not LIKE '%isMigrated%',isMigrated
1,queries\reports\v_autopay_profile.sql,136,"IF(cob.status_value LIKE '%isMigrated%', TRUE, FALSE) is_migrated_client,",isMigrated
2,queries\reports\v_autopay_to_reimbursement.sql,67,"IF(cob.status_value LIKE '%isMigrated%', TRUE, FALSE) is_migrated_client,",isMigrated
3,queries\reports\v_booked_deals_platform.sql,22,and cos.status_value not LIKE '%isMigrated%' -- only net new hra hub companies,isMigrated
4,queries\reports\v_company_onboarding.sql,15,"where JSON_EXTRACT_SCALAR(status_value, '$.isMigrated') = 'true')",isMigrated
5,queries\reports\v_ee_roster.sql,80,"IF(cob.status_value LIKE '%isMigrated%', true, false) is_migrated_client,",isMigrated
6,queries\reports\v_enrollment_app_qa.sql,138,"IF(cob.status_value LIKE '%isMigrated%', TRUE, FALSE) is_migrated_client,",isMigrated
7,queries\reports\v_env_rate_validation.sql,25,and cos2.status_value LIKE '%isMigrated%' and ac.`__hevo__marked_deleted` is not true),isMigrated
8,queries\reports\v_ye_reporting_contacts.sql,4,join env_prod.benefits_company_onboarding_status bcos on bcos.company_id = bc.id AND bcos.step = 'ACCOUNT' and bcos.`__hevo__marked_deleted` is not TRUE and bcos.status_value LIKE '%isMigrated%',isMigrated
9,queries\reports\v_ye_reporting_contacts.sql,96,bcos.status_value LIKE '%isMigrated%' migrated_to_hra_hub -- by default it is false,isMigrated


In [18]:
%pip install jinja2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
x ={'premium_amount_cents', 'enrollment_status', ' enrollment_type', 'is_primary', 'plan_effective_date', 'plan_end_date',
    'carrier_id', 'carrier_name', 'insurance_type', 'plan_market'}
y = {'id', 'created_at', 'created_by', 'updated_at', 'updated_by', 'carrier_id', 'carrier_name',
    'current_workflow_step_id', 'employment_id', 'enrollment_status', 'enrollment_type', 'health_plan_id', 
    'health_plan_name', 'health_plan_type', 'is_renewal', 'plan_effective_date', 'poc_document_id', 'premium_amount_cents', 
    'state', 'address_id', 'shopping_session_id', 'insurance_type', 'is_hsa_qualified', 'is_primary', 'plan_end_date', 'plan_market',
    'payment_details_id', 'source_health_benefit_election_id', 'renewal_health_plan_id', 'source_health_plan_id'}
y - x

{'address_id',
 'created_at',
 'created_by',
 'current_workflow_step_id',
 'employment_id',
 'enrollment_type',
 'health_plan_id',
 'health_plan_name',
 'health_plan_type',
 'id',
 'is_hsa_qualified',
 'is_renewal',
 'payment_details_id',
 'poc_document_id',
 'renewal_health_plan_id',
 'shopping_session_id',
 'source_health_benefit_election_id',
 'source_health_plan_id',
 'state',
 'updated_at',
 'updated_by'}